In [1]:
import asyncio
import datetime as dt
import math
from typing import Literal

import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np
import pandas as pd
import pandas_market_calendars as mcal
import plotly.graph_objects as go
import polars as pl
from dash import Dash, dcc, html
from plotly.subplots import make_subplots

nse = mcal.get_calendar("NSE")

In [2]:
pd.set_option("display.max_rows", 25_000)
pd.set_option("display.max_columns", 500)
pl.Config.set_tbl_cols(500)
pl.Config.set_tbl_rows(10_000)

pd.options.display.float_format = "{:.4f}".format

In [3]:
import sys

sys.path.append("..")
from tooling.enums import AssetClass, Index, Spot, StrikeSpread
from tooling.fetch import fetch_option_data, fetch_spot_data
from tooling.filter import find_atm, option_tool

In [4]:
async def get_expiry(f_today, index):

    if index == 'bnf':    
        if (f_today <= dt.date(2024, 1, 25)) and (f_today >= dt.date(2024, 1, 18)):
            f_expiry = dt.date(2024, 1, 25)
        elif (f_today <= dt.date(2024, 1, 31)) and (f_today >= dt.date(2024, 1, 26)):
            f_expiry = dt.date(2024, 1, 31)
        elif (f_today <= dt.date(2024, 2, 22)) and (f_today >= dt.date(2024, 2, 29)):
            f_expiry = dt.date(2024, 2, 29)
        elif (f_today <= dt.date(2024, 3, 25)) and (f_today >= dt.date(2024, 3, 27)):
            f_expiry = dt.date(2024, 2, 27)
        elif f_today < dt.date(2023, 9, 1):
            days_to_thursday = (3 - f_today.weekday()) % 7
            nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
            f_expiry = nearest_thursday
            if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
                f_expiry = nearest_thursday - dt.timedelta(days=1)
        elif f_today >= dt.date(2023, 9, 1):
            if f_today.day < 24:
                days_to_wednesday = (2 - f_today.weekday()) % 7
                nearest_wednesday = f_today + dt.timedelta(days=days_to_wednesday)
                f_expiry = nearest_wednesday
                if nse.valid_days(
                    start_date=nearest_wednesday, end_date=nearest_wednesday
                ).empty:
                    f_expiry = nearest_wednesday - dt.timedelta(days=1)
            else:
                days_to_thursday = (3 - f_today.weekday()) % 7
                nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
                f_expiry = nearest_thursday
                if nse.valid_days(
                    start_date=nearest_thursday, end_date=nearest_thursday
                ).empty:
                    f_expiry = nearest_thursday - dt.timedelta(days=1)
        return f_expiry

    elif index == 'nifty':
        days_to_thursday = (3 - f_today.weekday()) % 7
        nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
        f_expiry = nearest_thursday
        if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
            f_expiry = nearest_thursday - dt.timedelta(days=1)
        return f_expiry

    elif index == 'finnifty' or index == 'fnf':
        days_to_thursday = (1 - f_today.weekday()) % 7
        nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
        f_expiry = nearest_thursday
        if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
            f_expiry = nearest_thursday - dt.timedelta(days=1)
        return f_expiry

    elif index == 'midcpnifty' or index == 'midcp':
        days_to_thursday = (0 - f_today.weekday()) % 7
        nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
        f_expiry = nearest_thursday
        if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
            f_expiry = nearest_thursday - dt.timedelta(days=1)
        return f_expiry

async def get_expiry_nifty(f_today):

    days_to_thursday = (3 - f_today.weekday()) % 7
    nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
    f_expiry = nearest_thursday
    if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
        f_expiry = nearest_thursday - dt.timedelta(days=1)
    return f_expiry


async def get_option_contract_name(symbol, strike, expiry, opt_type):
    temp = "0"
    mth = expiry.month

    if (expiry + dt.timedelta(days=7)).month != expiry.month:
        date_string = expiry.strftime("%y%b").upper()
        return f"{symbol}{date_string}{strike}{opt_type}"
    else:
        if expiry.day <= 9:
            date_string = f"{expiry.year - 2000}{mth}{temp}{expiry.day}"
        else:
            date_string = f"{expiry.year - 2000}{mth}{expiry.day}"
        return f"{symbol}{date_string}{strike}{opt_type}"


def get_option_contract_name2(symbol, strike, expiry, opt_type):
    temp = "0"
    mth = expiry.month

    if (expiry + dt.timedelta(days=7)).month != expiry.month:
        date_string = expiry.strftime("%y%b").upper()
        return f"{symbol}{date_string}{strike}{opt_type}"
    else:
        if expiry.day <= 9:
            date_string = f"{expiry.year - 2000}{mth}{temp}{expiry.day}"
        else:
            date_string = f"{expiry.year - 2000}{mth}{expiry.day}"
        return f"{symbol}{date_string}{strike}{opt_type}"

In [5]:
bnf_1min = pd.read_csv("../data/sensex_30min_tv.csv")
bnf_1min["datetime"] = pd.to_datetime(bnf_1min["time"])
bnf_1min = bnf_1min[bnf_1min["datetime"].dt.year >= 2017]

In [6]:
bnf_1min.tail()

,time,open,high,low,close,datetime
22323,2024-12-18T13:15:00+05:30,80253.0900,80296.6400,80124.1400,80126.3500,2024-12-18 13:15:00+05:30
22324,2024-12-18T13:45:00+05:30,80128.2400,80255.3600,80104.3300,80169.5500,2024-12-18 13:45:00+05:30
22325,2024-12-18T14:15:00+05:30,80174.8200,80220.7400,80083.7900,80131.4000,2024-12-18 14:15:00+05:30
22326,2024-12-18T14:45:00+05:30,80131.6000,80215.7400,80103.1800,80159.2100,2024-12-18 14:45:00+05:30
22327,2024-12-18T15:15:00+05:30,80158.8000,80253.1700,80156.2800,80239.7800,2024-12-18 15:15:00+05:30


In [7]:
def resample(
    data: pl.DataFrame, timeframe, offset: dt.timedelta | None = None
) -> pl.DataFrame:
    return (
        data.set_sorted("datetime")
        .group_by_dynamic(
            index_column="datetime",
            every=timeframe,
            period=timeframe,
            label="left",
            offset=offset,
        )
        .agg(
            [
                pl.col("open").first().alias("open"),
                pl.col("high").max().alias("high"),
                pl.col("low").min().alias("low"),
                pl.col("close").last().alias("close"),
                pl.col("volume").sum().alias("volume"),
            ]
        )
    )


# ohlc_resampled = resample(pl.DataFrame(bnf_1min), '7d', pd.Timedelta(days=4))
# ohlc_resampled

In [8]:
bnf_1min["datetime"] = pd.to_datetime(bnf_1min["datetime"])
list_of_traded_dates = set(bnf_1min["datetime"].dt.date)
list_of_traded_dates

{datetime.date(2024, 3, 15),
 datetime.date(2018, 10, 4),
 datetime.date(2023, 7, 11),
 datetime.date(2020, 4, 9),
 datetime.date(2024, 10, 21),
 datetime.date(2018, 3, 21),
 datetime.date(2023, 9, 14),
 datetime.date(2022, 6, 6),
 datetime.date(2019, 10, 14),
 datetime.date(2019, 8, 22),
 datetime.date(2024, 7, 24),
 datetime.date(2019, 3, 5),
 datetime.date(2022, 2, 23),
 datetime.date(2023, 10, 20),
 datetime.date(2021, 3, 22),
 datetime.date(2020, 11, 14),
 datetime.date(2023, 9, 13),
 datetime.date(2024, 9, 24),
 datetime.date(2024, 9, 26),
 datetime.date(2018, 10, 9),
 datetime.date(2020, 4, 13),
 datetime.date(2020, 5, 22),
 datetime.date(2024, 2, 16),
 datetime.date(2022, 10, 11),
 datetime.date(2022, 11, 2),
 datetime.date(2020, 11, 19),
 datetime.date(2023, 4, 21),
 datetime.date(2020, 1, 30),
 datetime.date(2019, 4, 26),
 datetime.date(2019, 2, 25),
 datetime.date(2019, 2, 28),
 datetime.date(2019, 9, 6),
 datetime.date(2020, 6, 5),
 datetime.date(2024, 7, 3),
 datetime.date

In [19]:
import pandas as pd

def calculate_signals(df, n, rsi_period, rsi_overbought, ema_length):
    # Calculate RSI
    def calculate_rsi(series, period):
        # Calculate the price changes
        delta = series.diff()
    
        # Separate gains and losses
        gain = delta.where(delta > 0, 0)
        loss = -delta.where(delta < 0, 0)
    
        # Calculate the first average gain and loss (Simple Moving Average for the first period)
        avg_gain = gain.rolling(window=period, min_periods=period).mean()
        avg_loss = loss.rolling(window=period, min_periods=period).mean()
    
        # Calculate the subsequent smoothed averages (similar to EMA)
        for i in range(period, len(series)):
            avg_gain.iloc[i] = (avg_gain.iloc[i - 1] * (period - 1) + gain.iloc[i]) / period
            avg_loss.iloc[i] = (avg_loss.iloc[i - 1] * (period - 1) + loss.iloc[i]) / period
    
        # Calculate the Relative Strength (RS) and the RSI
        rs = avg_gain / avg_loss
        rsi = 100 - (100 / (1 + rs))
    
        return rsi

    # Apply the TradingView-style RSI calculation
    df['RSI'] = calculate_rsi(df['close'], rsi_period)

    # Calculate the previous n candles' low
    df['Prev_N_Low'] = df['low'].rolling(window=n).min().shift(1)
    df['ema'] = df['close'].ewm(span=ema_length, adjust=False).mean()

    # Signal generation logic
    df['Sell_Signal'] = (
        (df['close'] < df['Prev_N_Low']) &
        (df['RSI'] > rsi_overbought)
    )

    return df

In [20]:
bnf_1min.head()

,time,open,high,low,close,datetime,RSI,Prev_N_Low,ema,Sell_Signal
0,2018-01-01T09:15:00+05:30,34090.2600,34101.1300,34032.7000,34056.7200,2018-01-01 09:15:00+05:30,NaN,NaN,34056.7200,False
1,2018-01-01T09:45:00+05:30,34058.0400,34090.4300,34045.5700,34084.8200,2018-01-01 09:45:00+05:30,NaN,NaN,34058.2811,False
2,2018-01-01T10:15:00+05:30,34084.3900,34095.2200,34063.6000,34079.0000,2018-01-01 10:15:00+05:30,NaN,NaN,34059.4322,False
3,2018-01-01T10:45:00+05:30,34078.8700,34092.1300,34062.9700,34065.4600,2018-01-01 10:45:00+05:30,NaN,NaN,34059.7670,False
4,2018-01-01T11:15:00+05:30,34065.9700,34084.5700,34065.9700,34080.2400,2018-01-01 11:15:00+05:30,NaN,34032.7000,34060.9044,False


In [21]:
lev_ = 7
portfolio_value = 10000000
lot_size_ = 10
slippage_ = 0.0001

In [22]:
def backtest_short(df, n, rsi_period, rsi_overbought, ema_length):
    trade_book = []
    in_trade = False
    points = 0
    is_trailing_active = False
    trailing_sl = 0
    df = calculate_signals(df, n, rsi_period, rsi_overbought, ema_length)
    # print(df[df['Sell_Signal']].to_string())
    for i in range(n, len(df)):
        # print(i, df.loc[i], df.loc[i, 'open'])
        # break
        if df.loc[i, 'Sell_Signal'] and not in_trade:
            # Entry Triggered
            entry_price = df.loc[i, 'close']
            entry_time = df.loc[i, 'datetime']
            initial_sl = df.loc[i-n:i, 'high'].max()
            sl_in_points = initial_sl - entry_price
            target = 5 * sl_in_points
            in_trade = True

        if in_trade:
            if df.loc[i, 'high'] < df.loc[i, 'ema']:
                is_trailing_active = True
            if is_trailing_active:
                trailing_sl = df.loc[i, 'ema']
            # print(df.loc[i, 'open'], initial_sl)
            # break
            if df.loc[i, 'open'] > initial_sl and not points:
                # Gap condition
                if df.loc[i, 'close'] < initial_sl:
                    initial_sl = df.loc[i, 'high']
                else:
                    exit_price = df.loc[i, 'close']
                    exit_time = df.loc[i, 'datetime']
                    in_trade = False
                    remark = 'Gap SL'
                    points = entry_price - exit_price
            if df.loc[i, 'high'] > initial_sl and not points:
                # Initial SL Hit
                exit_price = df.loc[i, 'close']
                exit_time = df.loc[i, 'datetime']
                in_trade = False
                remark = 'Initial SL'
                points = entry_price - exit_price
            if is_trailing_active:
                if df.loc[i, 'close'] > trailing_sl and not points:
                    # TSL Hit
                    exit_price = df.loc[i, 'close']
                    exit_time = df.loc[i, 'datetime']
                    in_trade = False
                    remark = 'Trailing SL'
                    points = entry_price - exit_price
            if df.loc[i, 'low'] < entry_price - target and not points:
                # Target Hit
                exit_price = entry_price - target
                exit_time = df.loc[i, 'datetime']
                in_trade = False
                remark = 'Target'
                points = entry_price - exit_price

        if points:
            index_lev = lev_
            qty = int(round(portfolio_value * index_lev / entry_price / lot_size_)) * lot_size_
            slippage = slippage_ * (entry_price + exit_price)
            final_points = points - slippage
            trade = {
                "Trade Type": "Short",
                "Entry Time": entry_time,
                "Entry Price": entry_price,
                "Initial SL": initial_sl,
                "Trailing SL": trailing_sl,
                "Exit Time": exit_time,
                "Exit Price": exit_price,
                "Points Captured": points,
                "Slippage in Points": slippage,
                "After Costs": final_points,
                "PnL": final_points * qty,
                "Remarks": remark,
                "Qty": qty,
                "Leverage": index_lev,
                "ROI%": (final_points * qty / portfolio_value) * 100,
                "Trade Year": entry_time.year,
                "Trade Month": entry_time.month,
            }
            # print(trade)
            trade_book.append(trade)
            points = 0
            in_trade = False
            is_trailing_active = False
            trailing_sl = 0

    return pd.DataFrame(trade_book)
            

In [23]:
tb = backtest_short(bnf_1min, n=3, rsi_period=5, rsi_overbought=50, ema_length=14)

In [24]:
def generate_stats(tb_expiry, variation):
    stats_df8 = pd.DataFrame(
        index=range(2018, 2025),
        columns=[
            "Total ROI",
            "Total Trades",
            "Win Rate",
            "Avg Profit% per Trade",
            "Avg Loss% per Trade",
            "Max Drawdown",
            "ROI/DD Ratio",
            "Variation",
        ],
    )
    combined_df_sorted = tb_expiry
    # combined_df_sorted = tb_expiry_ce
    # combined_df_sorted = tb_expiry_pe
    
    # Iterate over each year
    for year in range(2018, 2025):
        # Filter trades for the current year
        year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]
    
        # Calculate total ROI
        total_roi = year_trades["ROI%"].sum()
    
        # Calculate total number of trades
        total_trades = len(year_trades)
    
        # Calculate win rate
        win_rate = (year_trades["ROI%"] > 0).mean() * 100
    
        # Calculate average profit per trade
        avg_profit = year_trades[year_trades["ROI%"] > 0]["ROI%"].mean()
    
        # Calculate average loss per trade
        avg_loss = year_trades[year_trades["ROI%"] < 0]["ROI%"].mean()
    
        # Calculate maximum drawdown
        max_drawdown = (
            year_trades["ROI%"].cumsum() - year_trades["ROI%"].cumsum().cummax()
        ).min()
    
        # Calculate ROI/DD ratio
        roi_dd_ratio = total_roi / abs(max_drawdown)

        variation = variation
    
        # Store the statistics in the DataFrame
        stats_df8.loc[year] = [
            total_roi,
            total_trades,
            win_rate,
            avg_profit,
            avg_loss,
            max_drawdown,
            roi_dd_ratio,
            variation,
        ]
    
    # Calculate overall statistics
    overall_total_roi = stats_df8["Total ROI"].sum()
    overall_total_trades = stats_df8["Total Trades"].sum()
    overall_win_rate = (combined_df_sorted["ROI%"] > 0).mean() * 100
    overall_avg_profit = combined_df_sorted[combined_df_sorted["ROI%"] > 0]["ROI%"].mean()
    overall_avg_loss = combined_df_sorted[combined_df_sorted["ROI%"] < 0]["ROI%"].mean()
    overall_max_drawdown = (
        combined_df_sorted["ROI%"].cumsum() - combined_df_sorted["ROI%"].cumsum().cummax()
    ).min()
    overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)
    overall_variation = variation
    
    # Store the overall statistics in the DataFrame
    stats_df8.loc["Overall"] = [
        overall_total_roi,
        overall_total_trades,
        overall_win_rate,
        overall_avg_profit,
        overall_avg_loss,
        overall_max_drawdown,
        overall_roi_dd_ratio,
        overall_variation,
    ]
    return {overall_roi_dd_ratio : stats_df8}

In [25]:
variation = 1
stats = generate_stats(tb, variation)
for x, y in stats.items():
    final_stats = y

final_stats

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio,Variation
2018,9.2925,29,41.3793,4.1902,-2.4112,-16.7676,0.5542,1
2019,19.7194,14,42.8571,5.1883,-1.4263,-4.4839,4.3979,1
2020,37.1651,18,50.0000,7.3510,-3.2216,-12.0663,3.0801,1
2021,22.1937,25,40.0000,5.9794,-2.5067,-14.5797,1.5222,1
2022,-23.0257,26,38.4615,2.9037,-3.2539,-33.5695,-0.6859,1
2023,-12.5765,20,40.0000,2.6096,-2.7878,-25.0836,-0.5014,1
2024,11.6129,12,50.0000,4.6761,-2.7406,-16.4436,0.7062,1
Overall,64.3815,144.0000,42.3611,4.6777,-2.6621,-62.1565,1.0358,1.0000


In [26]:
tb[tb['Trade Year'] > 2023]

,Trade Type,Entry Time,Entry Price,Initial SL,Trailing SL,Exit Time,Exit Price,Points Captured,Slippage in Points,After Costs,PnL,Remarks,Qty,Leverage,ROI%,Trade Year,Trade Month
132,Short,2024-01-05 10:45:00+05:30,71955.6900,72156.4800,0.0000,2024-01-08 09:15:00+05:30,71896.6600,59.0300,14.3852,44.6448,43305.4220,Initial SL,970,7,0.4331,2024,1
133,Short,2024-01-19 10:45:00+05:30,71604.6300,72026.2600,71664.7258,2024-01-23 09:15:00+05:30,71949.8100,-345.1800,14.3554,-359.5354,-352344.7351,Initial SL,980,7,-3.5234,2024,1
134,Short,2024-04-09 11:45:00+05:30,74956.1200,75123.3000,74785.3973,2024-04-10 09:15:00+05:30,74949.0500,7.0700,14.9905,-7.9205,-7366.0808,Trailing SL,930,7,-0.0737,2024,4
135,Short,2024-04-30 12:45:00+05:30,74865.3800,75010.6200,0.0000,2024-04-30 13:45:00+05:30,75065.2600,-199.8800,14.9931,-214.8731,-201980.6802,Initial SL,940,7,-2.0198,2024,4
136,Short,2024-06-06 11:45:00+05:30,74684.5900,75297.7300,0.0000,2024-06-07 09:15:00+05:30,75463.3100,-778.7200,15.0148,-793.7348,-746110.7026,Initial SL,940,7,-7.4611,2024,6
137,Short,2024-06-10 11:45:00+05:30,76643.2300,76884.2600,0.0000,2024-06-10 13:45:00+05:30,76844.0500,-200.8200,15.3487,-216.1687,-196713.5425,Initial SL,910,7,-1.9671,2024,6
138,Short,2024-07-03 14:15:00+05:30,79880.0000,80003.1700,0.0000,2024-07-03 14:45:00+05:30,80022.9200,-142.9200,15.9903,-158.9103,-139841.0570,Initial SL,880,7,-1.3984,2024,7
139,Short,2024-07-19 09:15:00+05:30,81272.1900,81587.7600,80541.6812,2024-07-23 10:45:00+05:30,80664.8100,607.3800,16.1937,591.1863,508420.2180,Trailing SL,860,7,5.0842,2024,7
140,Short,2024-08-22 12:45:00+05:30,81050.2400,81171.4200,0.0000,2024-08-23 09:15:00+05:30,80928.1600,122.0800,16.1978,105.8822,91058.6576,Initial SL,860,7,0.9106,2024,8
141,Short,2024-10-28 13:45:00+05:30,80178.4800,80539.8100,79858.7435,2024-10-29 11:45:00+05:30,79897.6400,280.8400,16.0076,264.8324,230404.1776,Trailing SL,870,7,2.3040,2024,10


In [27]:
stats_dictionary = {}
rsi_lengths = [14, 9, 21, 6, 12, 16]
for i in range(2, 5):
    for j in rsi_lengths:
        for k in range(30, 81, 5):
            for l in range(5, 36, 5):
                variation = f'n {i}, rsi {j}, threshold {k}, ema {l}'
                print(variation)
                tb = backtest_short(bnf_1min, i, j, k, l)
                if len(tb) > 0:
                    stats = generate_stats(tb, variation)
                    for x, y in stats.items():
                        if x > 8:
                            final_stats = y
                            print(final_stats.to_string())
                            stats_dictionary[x] = y


n 2, rsi 14, threshold 30, ema 5
n 2, rsi 14, threshold 30, ema 10
n 2, rsi 14, threshold 30, ema 15
n 2, rsi 14, threshold 30, ema 20
n 2, rsi 14, threshold 30, ema 25
n 2, rsi 14, threshold 30, ema 30
n 2, rsi 14, threshold 30, ema 35
n 2, rsi 14, threshold 35, ema 5
n 2, rsi 14, threshold 35, ema 10
n 2, rsi 14, threshold 35, ema 15
n 2, rsi 14, threshold 35, ema 20
n 2, rsi 14, threshold 35, ema 25
n 2, rsi 14, threshold 35, ema 30
n 2, rsi 14, threshold 35, ema 35
n 2, rsi 14, threshold 40, ema 5
n 2, rsi 14, threshold 40, ema 10
n 2, rsi 14, threshold 40, ema 15
n 2, rsi 14, threshold 40, ema 20
n 2, rsi 14, threshold 40, ema 25
n 2, rsi 14, threshold 40, ema 30
n 2, rsi 14, threshold 40, ema 35
n 2, rsi 14, threshold 45, ema 5
n 2, rsi 14, threshold 45, ema 10
n 2, rsi 14, threshold 45, ema 15
n 2, rsi 14, threshold 45, ema 20
n 2, rsi 14, threshold 45, ema 25
n 2, rsi 14, threshold 45, ema 30
n 2, rsi 14, threshold 45, ema 35
n 2, rsi 14, threshold 50, ema 5
n 2, rsi 14, thresh

/tmp/ipykernel_36346/873016624.py:45: RuntimeWarning: divide by zero encountered in scalar divide
  roi_dd_ratio = total_roi / abs(max_drawdown)


n 2, rsi 14, threshold 75, ema 10


/tmp/ipykernel_36346/873016624.py:45: RuntimeWarning: divide by zero encountered in scalar divide
  roi_dd_ratio = total_roi / abs(max_drawdown)


n 2, rsi 14, threshold 75, ema 15


...

n 4, rsi 16, threshold 70, ema 30


/tmp/ipykernel_36346/873016624.py:45: RuntimeWarning: divide by zero encountered in scalar divide
  roi_dd_ratio = total_roi / abs(max_drawdown)


n 4, rsi 16, threshold 70, ema 35


/tmp/ipykernel_36346/873016624.py:45: RuntimeWarning: divide by zero encountered in scalar divide
  roi_dd_ratio = total_roi / abs(max_drawdown)


n 4, rsi 16, threshold 75, ema 5
n 4, rsi 16, threshold 75, ema 10
n 4, rsi 16, threshold 75, ema 15
n 4, rsi 16, threshold 75, ema 20
n 4, rsi 16, threshold 75, ema 25
n 4, rsi 16, threshold 75, ema 30
n 4, rsi 16, threshold 75, ema 35
n 4, rsi 16, threshold 80, ema 5
n 4, rsi 16, threshold 80, ema 10
n 4, rsi 16, threshold 80, ema 15
n 4, rsi 16, threshold 80, ema 20
n 4, rsi 16, threshold 80, ema 25
n 4, rsi 16, threshold 80, ema 30
n 4, rsi 16, threshold 80, ema 35


In [18]:
stats_dictionary

{}